In [3]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output

# Sample asset data: location (x, y coordinates), condition (good, bad, maintenance)
data = {
    'asset_id': ['A1', 'A2', 'A3', 'A4', 'A5'],
    'x': [5, 15, 25, 35, 45],
    'y': [10, 20, 10, 40, 30],
    'condition': ['good', 'bad', 'maintenance', 'good', 'bad'],
    'details': ['This is asset A1', 'This is asset A2', 'This is asset A3', 'This is asset A4', 'This is asset A5']
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Create the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Asset States on Floor Map"),
    dcc.Graph(id='asset-map'),
])

# Define the callback to update the graph based on the dataset
@app.callback(
    Output('asset-map', 'figure'),
    Input('asset-map', 'id')  # The input here doesn't change, so we just use a static Input
)
def update_graph(_):
    # Create the Plotly figure
    fig = px.scatter(
        df,
        x='x',
        y='y',
        color='condition',
        text='asset_id',
        hover_name='asset_id',  # Add asset ID to the hover
        hover_data={'condition': True, 'x': True, 'y': True, 'details': True},  # Show extra details on hover
        title='Asset States on Floor Map (Interactive)',
        labels={'x': 'X Coordinate', 'y': 'Y Coordinate'},  # Axis labels
    )

    # Update marker style and hover behavior
    fig.update_traces(
        marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')),  # Marker style
        selector=dict(mode='markers+text'),  # Show both markers and labels
        textposition='top center'  # Position labels
    )

    # Enable zoom, pan, and hover interactions
    fig.update_layout(
        hovermode='closest',  # Show details on hover for the closest point
        dragmode='pan',  # Enable panning across the plot
        title_x=0.5,  # Center the title
        xaxis=dict(range=[0, 50], fixedrange=False),  # Allow zooming on x-axis
        yaxis=dict(range=[0, 50], fixedrange=False),  # Allow zooming on y-axis
    )

    return fig

# Run the Dash app in a new browser window
if __name__ == '__main__':
    app.run_server(debug=True)


In [7]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output, State

# Sample asset data: location (x, y coordinates), condition (good, bad, maintenance)
data = {
    'asset_id': ['A1', 'A2', 'A3', 'A4', 'A5'],
    'x': [5, 15, 25, 35, 45],
    'y': [10, 20, 10, 40, 30],
    'condition': ['good', 'bad', 'maintenance', 'good', 'bad'],
    'details': ['This is asset A1', 'This is asset A2', 'This is asset A3', 'This is asset A4', 'This is asset A5']
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Create the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Interactive Asset Management Dashboard"),
    
    # Search Bar for assets
    html.Div([
        html.Label("Search by Asset ID:"),
        dcc.Input(id='asset-search', type='text', placeholder='Enter Asset ID (e.g., A1)', debounce=True),
    ], style={'padding': '10px', 'width': '50%'}),

    # Dropdown for asset condition filtering (multiselect)
    html.Div([
        html.Label("Filter by Asset Condition:"),
        dcc.Dropdown(
            id='condition-filter',
            options=[
                {'label': 'Good', 'value': 'good'},
                {'label': 'Bad', 'value': 'bad'},
                {'label': 'Maintenance', 'value': 'maintenance'}
            ],
            value=['good', 'bad', 'maintenance'],  # Default is showing all conditions
            multi=True,  # Allow multiple selection
            clearable=False
        ),
    ], style={'width': '50%', 'display': 'inline-block'}),

    # Range slider for X and Y coordinates
    html.Div([
        html.Label("Adjust X and Y Coordinates:"),
        dcc.RangeSlider(
            id='x-slider',
            min=0,
            max=50,
            step=1,
            value=[0, 50],  # Initial range for X-axis
            marks={i: str(i) for i in range(0, 51, 5)},  # Markers every 5 units
            tooltip={"placement": "bottom", "always_visible": True}
        ),
        dcc.RangeSlider(
            id='y-slider',
            min=0,
            max=50,
            step=1,
            value=[0, 50],  # Initial range for Y-axis
            marks={i: str(i) for i in range(0, 51, 5)},  # Markers every 5 units
            tooltip={"placement": "bottom", "always_visible": True}
        )
    ], style={'width': '98%', 'padding': '20px', 'display': 'inline-block'}),
    
    # Asset Status Update Section
    html.Div([
        html.H3("Update Asset Status:"),
        dcc.Dropdown(
            id='asset-update-dropdown',
            options=[{'label': asset, 'value': asset} for asset in df['asset_id']],
            placeholder="Select Asset ID to Update",
            clearable=False
        ),
        dcc.Dropdown(
            id='status-update-dropdown',
            options=[
                {'label': 'Good', 'value': 'good'},
                {'label': 'Bad', 'value': 'bad'},
                {'label': 'Maintenance', 'value': 'maintenance'}
            ],
            placeholder="Select New Status",
            clearable=False
        ),
        html.Button('Update Status', id='update-button', n_clicks=0),
        html.Div(id='update-output', style={'padding': '10px'})
    ], style={'padding': '20px', 'border': '1px solid #ccc'}),

    # Graph showing the assets
    dcc.Graph(id='asset-map'),

])

# Callback for updating the graph based on search, filter, and coordinate range
@app.callback(
    Output('asset-map', 'figure'),
    [Input('condition-filter', 'value'),
     Input('x-slider', 'value'),
     Input('y-slider', 'value'),
     Input('asset-search', 'value')]
)
def update_graph(selected_conditions, x_range, y_range, search_query):
    # Filter by condition and coordinates
    filtered_df = df[df['condition'].isin(selected_conditions)]
    filtered_df = filtered_df[(filtered_df['x'] >= x_range[0]) & (filtered_df['x'] <= x_range[1])]
    filtered_df = filtered_df[(filtered_df['y'] >= y_range[0]) & (filtered_df['y'] <= y_range[1])]

    # If there's a search query, filter by asset ID
    if search_query:
        filtered_df = filtered_df[filtered_df['asset_id'].str.contains(search_query, case=False)]

    # Create the plot
    fig = px.scatter(
        filtered_df,
        x='x',
        y='y',
        color='condition',
        text='asset_id',
        hover_name='asset_id',  # Add asset ID to the hover
        hover_data={'condition': True, 'x': True, 'y': True, 'details': True},  # Show extra details on hover
        title='Asset States on Floor Map (Interactive)',
        labels={'x': 'X Coordinate', 'y': 'Y Coordinate'},  # Axis labels
    )

    # Update marker style and hover behavior
    fig.update_traces(
        marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')),  # Marker style
        selector=dict(mode='markers+text'),  # Show both markers and labels
        textposition='top center'  # Position labels
    )

    # Set layout properties
    fig.update_layout(
        hovermode='closest',  # Show details on hover for the closest point
        dragmode='pan',  # Enable panning across the plot
        title_x=0.5,  # Center the title
        xaxis=dict(range=[x_range[0], x_range[1]], fixedrange=False),  # Allow zooming on x-axis
        yaxis=dict(range=[y_range[0], y_range[1]], fixedrange=False),  # Allow zooming on y-axis
    )

    return fig

# Callback for updating the asset status
@app.callback(
    [Output('update-output', 'children'),
     Output('asset-update-dropdown', 'options')],
    [Input('update-button', 'n_clicks')],
    [State('asset-update-dropdown', 'value'),
     State('status-update-dropdown', 'value')]
)
def update_asset_status(n_clicks, selected_asset, new_status):
    if n_clicks > 0 and selected_asset and new_status:
        # Simulate database update by updating the DataFrame
        df.loc[df['asset_id'] == selected_asset, 'condition'] = new_status
        return (f"Updated {selected_asset} status to {new_status}.", 
                [{'label': asset, 'value': asset} for asset in df['asset_id']])
    return "", [{'label': asset, 'value': asset} for asset in df['asset_id']]

# Run the Dash app in a new browser window
if __name__ == '__main__':
    app.run_server(debug=True)


In [15]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output, State

# Sample asset data: location (x, y coordinates), condition (good, bad, maintenance)
data = {
    'asset_id': ['A1', 'A2', 'A3', 'A4', 'A5'],
    'x': [5, 15, 25, 35, 45],
    'y': [10, 20, 10, 40, 30],
    'condition': ['good', 'bad', 'maintenance', 'good', 'bad'],
    'details': ['This is asset A1', 'This is asset A2', 'This is asset A3', 'This is asset A4', 'This is asset A5']
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Create the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Interactive Asset Management Dashboard"),
    
    # Search Bar for assets
    html.Div([
        html.Label("Search by Asset ID:"),
        dcc.Input(id='asset-search', type='text', placeholder='Enter Asset ID (e.g., A1)', debounce=True),
    ], style={'padding': '10px', 'width': '50%'}),

    # Dropdown for asset condition filtering (multiselect)
    html.Div([
        html.Label("Filter by Asset Condition:"),
        dcc.Dropdown(
            id='condition-filter',
            options=[
                {'label': 'Good', 'value': 'good'},
                {'label': 'Bad', 'value': 'bad'},
                {'label': 'Maintenance', 'value': 'maintenance'}
            ],
            value=['good', 'bad', 'maintenance'],  # Default is showing all conditions
            multi=True,  # Allow multiple selection
            clearable=False
        ),
    ], style={'width': '50%', 'display': 'inline-block'}),

    # Range slider for X and Y coordinates
    html.Div([
        html.Label("Adjust X and Y Coordinates:"),
        dcc.RangeSlider(
            id='x-slider',
            min=0,
            max=50,
            step=1,
            value=[0, 50],  # Initial range for X-axis
            marks={i: str(i) for i in range(0, 51, 5)},  # Markers every 5 units
            tooltip={"placement": "bottom", "always_visible": True}
        ),
        dcc.RangeSlider(
            id='y-slider',
            min=0,
            max=50,
            step=1,
            value=[0, 50],  # Initial range for Y-axis
            marks={i: str(i) for i in range(0, 51, 5)},  # Markers every 5 units
            tooltip={"placement": "bottom", "always_visible": True}
        )
    ], style={'width': '98%', 'padding': '20px', 'display': 'inline-block'}),
    
    # Asset Status Update Section
    html.Div([
        html.H3("Update Asset Status:"),
        dcc.Dropdown(
            id='asset-update-dropdown',
            options=[{'label': asset, 'value': asset} for asset in df['asset_id']],
            placeholder="Select Asset ID to Update",
            clearable=False
        ),
        dcc.Dropdown(
            id='status-update-dropdown',
            options=[
                {'label': 'Good', 'value': 'good'},
                {'label': 'Bad', 'value': 'bad'},
                {'label': 'Maintenance', 'value': 'maintenance'}
            ],
            placeholder="Select New Status",
            clearable=False
        ),
        html.Button('Update Status', id='update-button', n_clicks=0),
        html.Div(id='update-output', style={'padding': '10px'})
    ], style={'padding': '20px', 'border': '1px solid #ccc'}),

    # Add New Asset Section
    html.Div([
        html.H3("Add New Asset:"),
        dcc.Input(id='new-asset-id', type='text', placeholder='Enter New Asset ID (e.g., A6)', debounce=True),
        dcc.Input(id='new-asset-x', type='number', placeholder='Enter X Coordinate', min=0, max=50, step=1),
        dcc.Input(id='new-asset-y', type='number', placeholder='Enter Y Coordinate', min=0, max=50, step=1),
        dcc.Dropdown(
            id='new-asset-condition',
            options=[
                {'label': 'Good', 'value': 'good'},
                {'label': 'Bad', 'value': 'bad'},
                {'label': 'Maintenance', 'value': 'maintenance'}
            ],
            placeholder="Select Condition",
            clearable=False
        ),
        html.Button('Add Asset', id='add-asset-button', n_clicks=0),
        html.Div(id='add-asset-output', style={'padding': '10px'})
    ], style={'padding': '20px', 'border': '1px solid #ccc'}),

    # Graph showing the assets
    dcc.Graph(id='asset-map'),
])

# Callback for updating the graph based on search, filter, and coordinate range
@app.callback(
    Output('asset-map', 'figure'),
    [Input('condition-filter', 'value'),
     Input('x-slider', 'value'),
     Input('y-slider', 'value'),
     Input('asset-search', 'value')]
)
def update_graph(selected_conditions, x_range, y_range, search_query):
    # Filter by condition and coordinates
    filtered_df = df[df['condition'].isin(selected_conditions)]
    filtered_df = filtered_df[(filtered_df['x'] >= x_range[0]) & (filtered_df['x'] <= x_range[1])]
    filtered_df = filtered_df[(filtered_df['y'] >= y_range[0]) & (filtered_df['y'] <= y_range[1])]

    # If there's a search query, filter by asset ID
    if search_query:
        filtered_df = filtered_df[filtered_df['asset_id'].str.contains(search_query, case=False)]

    # Create the plot
    fig = px.scatter(
        filtered_df,
        x='x',
        y='y',
        color='condition',
        text='asset_id',
        hover_name='asset_id',  # Add asset ID to the hover
        hover_data={'condition': True, 'x': True, 'y': True, 'details': True},  # Show extra details on hover
        title='Asset States on Floor Map (Interactive)',
        labels={'x': 'X Coordinate', 'y': 'Y Coordinate'},  # Axis labels
    )

    # Update marker style and hover behavior
    fig.update_traces(
        marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')),  # Marker style
        selector=dict(mode='markers+text'),  # Show both markers and labels
        textposition='top center'  # Position labels
    )

    # Set layout properties
    fig.update_layout(
        hovermode='closest',  # Show details on hover for the closest point
        dragmode='pan',  # Enable panning across the plot
        title_x=0.5,  # Center the title
        xaxis=dict(range=[x_range[0], x_range[1]], fixedrange=False),  # Allow zooming on x-axis
        yaxis=dict(range=[y_range[0], y_range[1]], fixedrange=False),  # Allow zooming on y-axis
    )

    return fig

# Callback for updating the asset status
@app.callback(
    [Output('update-output', 'children'),
     Output('asset-update-dropdown', 'options')],
    [Input('update-button', 'n_clicks')],
    [State('asset-update-dropdown', 'value'),
     State('status-update-dropdown', 'value')]
)
def update_asset_status(n_clicks, selected_asset, new_status):
    if n_clicks > 0 and selected_asset and new_status:
        # Simulate database update by updating the DataFrame
        df.loc[df['asset_id'] == selected_asset, 'condition'] = new_status
        return (f"Updated {selected_asset} status to {new_status}.", 
                [{'label': asset, 'value': asset} for asset in df['asset_id']])
    return "", [{'label': asset, 'value': asset} for asset in df['asset_id']]

# Callback for adding a new asset
@app.callback(
    [Output('add-asset-output', 'children'),
     Output('asset-update-dropdown', 'options')],
    [Input('add-asset-button', 'n_clicks')],
    [State('new-asset-id', 'value'),
     State('new-asset-x', 'value'),
     State('new-asset-y', 'value'),
     State('new-asset-condition', 'value')]
)
def add_new_asset(n_clicks, new_asset_id, new_x, new_y, new_condition):
    if n_clicks > 0 and new_asset_id and new_x is not None and new_y is not None and new_condition:
        # Check if the asset ID already exists
        if new_asset_id in df['asset_id'].values:
            return f"Asset ID {new_asset_id} already exists.", [{'label': asset, 'value': asset} for asset in df['asset_id']]

        # Add the new asset to the DataFrame
        new_asset = {'asset_id': new_asset_id, 'x': new_x, 'y': new_y, 'condition': new_condition, 'details': f"This is asset {new_asset_id}"}
        df.loc[len(df)] = new_asset
        
        return f"Added new asset {new_asset_id} successfully.", [{'label': asset, 'value': asset} for asset in df['asset_id']]
    
    return "", [{'label': asset, 'value': asset} for asset in df['asset_id']]

# Run the Dash app in a new browser window
if __name__ == '__main__':
    app.run_server(debug=True)
